In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import matplotlib.ticker as ticker
plt.rcParams['axes.unicode_minus'] = False
import random as random
import seaborn as sns
sns.set()
from matplotlib import style
style.use('ggplot')
%matplotlib inline
from scipy import stats
from scipy.stats import mannwhitneyu
from statsmodels.stats import weightstats as stests

# Normality Tests

In [ ]:
# Normality tests on the focused groups

alpha = 1e-3
columns = Indecrease_matched.columns
no_normal_dist_list = []
normal_dist_list = []

def stat_test_continuous_norm(start, end):
    """
    input: start and end should be integers, corresponding to the column number in excel sheet of data
    """
    dic = dict()
    name = ['test name', 'Increase normality p', 'Decrease normality p','both normal']
    for i in range(4):
        dic[name[i]] = []    
    
    for i in range(start,end):
        fat_data = increase.iloc[:,i].dropna()
        norm_data = decrease.iloc[:,i].dropna()
    
        if len(fat_data)<8 or len(norm_data)<8:
            continue
        else:
            score_fat, p_fat = stats.normaltest(fat_data)
            score_norm, p_norm = stats.normaltest(norm_data)
            
        if p_fat >= alpha and p_norm >= alpha:
            dic[name[0]].append(columns[i])
            dic[name[1]].append(round(p_fat,3))
            dic[name[2]].append(round(p_norm,3))
            dic[name[3]].append('Yes')
            
        else:
            dic[name[0]].append(columns[i])
            dic[name[1]].append(round(p_fat,3))
            dic[name[2]].append(round(p_norm,3))
            dic[name[3]].append('No')

        df = pd.DataFrame.from_dict(dic, orient='index')
        df = df.transpose()
        #df = df.sort_values(['stat p'],ascending=False).reset_index(drop=True)
    return df

norm_result = stat_test_continuous_norm(356,493)

# Z tests

In [ ]:
# Calculate Z test, lab selection is done in excel

def z_test(start,end):
    z = []
    p = []
    col_n = []
    for i in range(start-1,end):
        ztest ,pval = stests.ztest(fat.iloc[:,i].dropna(), x2=normal.iloc[:,i].dropna())
        z.append(round(ztest,2))
        p.append(round(pval,2))
        col_n.append(column_list[i])
    df = pd.DataFrame({'col_n':col_n, 'z':z, 'p':p})
    return df

z_test(356,493)

# Adding Case/Control Column

In [196]:
def addtreatmentcol(datasets):
    for dataset in datasets:
        dataset['CaseControl'] = np.where(dataset['state'] == 'Decrease_norm', 1, 0)
addtreatmentcol([b_o_2,b_g_2,b_r_2])

# Mann Whitney U tests on three different years

In [217]:
alpha = 1e-3
columns = b_o_g_r_2_full.columns
no_normal_dist_list = []
normal_dist_list = []
def stat_test_continuous_mann_year(start, end):
    """
    input: start and end should be integers, corresponding to the column number in excel sheet of data
    """
    dic = dict()
    name = ['test name', 'year0 p' , 'year5 p', 'year10 p']
    for i in range(4):
        dic[name[i]] = [] 
        
    for i in range(start,end):
        count = 0
        dic[name[count]].append(columns[i])
        for year in [0,5,10]:
            count += 1
            increase_year = increase_gr[increase_gr['time_lapse']==year]
            decrease_year = decrease_bo[decrease_bo['time_lapse']==year]
            
            fat_data = increase_year.iloc[:,i].dropna()
            norm_data = decrease_year.iloc[:,i].dropna()
    
            if norm_data.isin([0]).all() & fat_data.isin([0]).all():
                dic[name[count]].append('ns')
            else:
                score, p = mannwhitneyu(fat_data, norm_data)
                dic[name[count]].append(round(p,2))


    df = pd.DataFrame.from_dict(dic, orient='index')
    df = df.transpose()
    #df = df.sort_values(['mann p'],ascending=False).reset_index(drop=True)
    return df


# Correlation Distributions

In [2]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)


def corr_dist(parameter_list,dataset):
    fig, ax = plt.subplots(4,3,sharex=False,sharey=False,figsize = (18,15))
    c1 = 0
    c2 = 0
    
    for parameter in parameter_list:
        sample = dataset[dataset['BMI'].notna()]
        sample = sample[sample[parameter].notna()]
        sample = sample.sort_values(["pid","age"],ascending=True)
    
        pid_listPerParameter=[]
        for pid in sample['pid']:
            if pid not in pid_listPerParameter :
                pid_listPerParameter.append(pid)

        corr_list = []
        for i in pid_listPerParameter:
            person = sample[sample['pid']== i]    
            y = np.array(person['BMI'])
            z = np.array(person[parameter])

            dif_y = difference(y)
            dif_z = difference(z)

            correlation = dif_y.corr(dif_z)
            corr_list.append(correlation)

        ax[c1,c2].set_xlim([-1.5,1.5])
        sns.distplot(corr_list, ax=ax[c1,c2],kde=False, hist=True, bins='auto', color = 'darkblue', hist_kws={'edgecolor':'black'},kde_kws={'linewidth': 2, "label": "KDE"})
        ax[c1,c2].set_ylabel('')
        plt.setp(ax[c1,c2], xlabel='Correlation Value')
        plt.setp(ax[c1,c2], ylabel= parameter+' population')
        
        c2 += 1
        if c2%3 == 0:
            c1 += 1
            c2 = 0

# Kolmogorov–Smirnov test

In [ ]:
def ks_test(parameter_list,datasets):
    stat_list = []
    p_list = []
    for parameter in parameter_list:
        count = 0
        for dataset in datasets:
            
            sample = dataset[dataset['BMI'].notna()]
            sample = sample[sample[parameter].notna()]
            sample = sample.sort_values(["pid","age"],ascending=True)

            pid_listPerParameter=[]
            for pid in sample['pid']:
                if pid not in pid_listPerParameter :
                    pid_listPerParameter.append(pid)

            corr_list = []
            for i in pid_listPerParameter:
                person = sample[sample['pid']== i]    
                y = np.array(person['BMI'])
                z = np.array(person[parameter])

                dif_y = difference(y)
                dif_z = difference(z)

                correlation = dif_y.corr(dif_z)
                corr_list.append(correlation)
                
            if count == 0:
                count+=1
                data1 = corr_list
            elif count == 1:
                ks_stat, ks_p = stats.ks_2samp(data1, corr_list)
        
        stat_list.append(round(ks_stat,2))
        p_list.append(round(ks_p,2))

    return stat_list, p_list